# Audio augmentation with audiomentations

In [ ]:
 '''
Title:
    FinalCodeAudioAugm.ipynb
Version:
    0
Author(s):
    Nick
Creation Date:
    ?
Update(s):
    Date       | Changes
    22.12.2022 | 
 '''

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import librosa
import librosa.display
import librosa as lr
import tensorflow as tf
import tensorflow_io as tfio
import os
import soundfile as sf
from glob import glob

#audio_file = "/Users/nickjonas/Desktop/Aufnahmen für Uni/TestAugmentedGuitar/ReadAudio/ZOOM0020_LR_edited.wav"
audio_path = "/Users/nickjonas/Downloads/Aufnahmen für Uni/AugmentationFeuerwehr/1stTry mit 15 augm pro sample/AudioAugmREAD"
dest_path = "/Users/nickjonas/Downloads/Aufnahmen für Uni/AugmentationFeuerwehr/1stTry mit 15 augm pro sample/AudioAugmWRITE/Warntag Alarm"
os.chdir(audio_path)
audio_files = os.listdir()
#y, sr = librosa.load(audio_file, sr=44100)
#time = np.arange(0, len(y))/sr
print(audio_files)

['Warntag_edited05.wav', 'Warntag_edited10.wav', 'Warntag_edited04.wav', 'Warntag_edited06.wav', 'Warntag_edited07.wav', 'Warntag_edited03.wav', '.DS_Store', 'Warntag_edited02.wav', 'Warntag_edited01.wav', 'Warntag_edited09.wav', 'Warntag_edited08.wav']


In [3]:
from audiomentations import Compose, PitchShift, AirAbsorption, AddGaussianNoise, HighPassFilter, ClippingDistortion, LowPassFilter  

for audio_file in audio_files:
    try:
        audio_file_path = os.path.join(audio_path, audio_file)
        y, sr = librosa.load(audio_file_path, sr=44100)
        time = np.arange(0, len(y))/sr

        count = -1 #counts whole code (for variety of augmentations)
        currentLoop = 0 #counts filename change
        while count <= 15:

            augment = Compose([
                
                PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
                AirAbsorption(min_temperature=10.0, max_temperature=20.0, min_humidity=50.0, max_humidity=90.0, min_distance=1.0, max_distance=15.0, p=0.5),
                AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
                HighPassFilter(min_cutoff_freq=90.0, max_cutoff_freq=150.0, min_rolloff=12., max_rolloff=24., zero_phase=False, p=0.5),
                ClippingDistortion(min_percentile_threshold=0, max_percentile_threshold=40, p=0.5),
                LowPassFilter(min_cutoff_freq=2000.0, max_cutoff_freq=7500.0, min_rolloff=12., max_rolloff=24., zero_phase=False, p=0.5),
            ])

            if __name__ == "__name__":
                y, sr = librosa.load(audio_file, sr=44100)
            # Augment/transform/perturb the audio data
            augmented_y = augment(y, sr)

            while currentLoop < count: #current loop only counts up if its smaller than count (so it waits for the next augmentation and then changes the filename)
                currentLoop = currentLoop + 1
                filename = os.path.join(dest_path, audio_file)
                filename += str(currentLoop) + ".wav"
                sf.write (filename, augmented_y, sr)
            count = count + 1
    except:
        print("Nope")    

/Users/nickjonas/opt/anaconda3/envs/audio_data_science/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


Nope


In [5]:
import os
import random
from pathlib import Path

import numpy as np
import time
from scipy.io import wavfile
from tqdm import tqdm

from audiomentations import (
    AddGaussianNoise,
    TimeStretch,
    PitchShift,
    Shift,
    Normalize,
    TimeMask,
    AddGaussianSNR,
    Resample,
    ClippingDistortion,
    AddBackgroundNoise,
    AddShortNoises,
    PeakingFilter,
    PolarityInversion,
    Gain,
    Mp3Compression,
    LoudnessNormalization,
    Trim,
    LowPassFilter,
    LowShelfFilter,
    HighShelfFilter,
    HighPassFilter,
    BandPassFilter,
    ApplyImpulseResponse,
    Reverse,
    RoomSimulator,
    TanhDistortion,
    Compose,
    SomeOf,
    OneOf,
    BandStopFilter,
    GainTransition,
    Padding,
    AirAbsorption,
    Lambda,
)
from audiomentations.augmentations.limiter import Limiter
from audiomentations.augmentations.seven_band_parametric_eq import SevenBandParametricEQ
from audiomentations.core.audio_loading_utils import load_sound_file
from audiomentations.core.transforms_interface import (
    MultichannelAudioNotSupportedException,
)

DEMO_DIR = os.path.dirname(__file__)


class timer(object):
    """
    timer: A class used to measure the execution time of a block of code that is
    inside a "with" statement.
    Example:
    ```
    with timer("Count to 500000"):
        x = 0
        for i in range(500000):
            x += 1
        print(x)
    ```
    Will output:
    500000
    Count to 500000: 0.04 s
    Warning: The time resolution used here may be limited to 1 ms
    """

    def __init__(self, description="Execution time", verbose=False):
        self.description = description
        self.verbose = verbose
        self.execution_time = None

    def __enter__(self):
        self.t = time.time()
        return self

    def __exit__(self, type, value, traceback):
        self.execution_time = time.time() - self.t
        if self.verbose:
            print("{}: {:.3f} s".format(self.description, self.execution_time))


if __name__ == "__main__":
    """
    For each transformation, apply it to an example sound and write the transformed sounds to
    an output folder. Also crudely measure and print execution time.
    """
    output_dir = os.path.join(DEMO_DIR, "output")
    os.makedirs(output_dir, exist_ok=True)

    np.random.seed(420)
    random.seed(420)

    sound_file_paths = [
        Path(os.path.join(DEMO_DIR, "acoustic_guitar_0.wav")),
        Path(os.path.join(DEMO_DIR, "perfect-alley1.ogg")),
        Path(os.path.join(DEMO_DIR, "p286_011.wav")),
    ]

    transforms = [
        {
            "instance": AddBackgroundNoise(
                sounds_path=os.path.join(DEMO_DIR, "background_noises"), p=1.0
            ),
            "num_runs": 5,
            "name": "AddBackgroundNoiseRelative",
        },
        {
            "instance": AddBackgroundNoise(
                sounds_path=os.path.join(DEMO_DIR, "background_noises"),
                noise_rms="absolute",
                min_absolute_rms_in_db=-30,
                max_absolute_rms_in_db=-10,
                p=1.0,
            ),
            "num_runs": 5,
            "name": "AddBackgroundNoiseAbsolute",
        },
        {
            "instance": AddBackgroundNoise(
                sounds_path=os.path.join(DEMO_DIR, "background_noises"),
                min_snr_in_db=2,
                max_snr_in_db=4,
                noise_transform=Reverse(p=1.0),
                p=1.0,
            ),
            "num_runs": 5,
            "name": "AddBackgroundNoiseWithTransform",
        },
        {
            "instance": AddGaussianNoise(
                min_amplitude=0.001, max_amplitude=0.015, p=1.0
            ),
            "num_runs": 5,
        },
        {
            "instance": AddGaussianSNR(p=1.0),
            "num_runs": 5,
            "name": "AddGaussianSNR",
        },
        {
            "instance": ApplyImpulseResponse(
                p=1.0,
                ir_path=os.path.join(DEMO_DIR, "ir"),
                leave_length_unchanged=False,
            ),
            "num_runs": 1,
            "name": "ApplyImpulseResponseWithTail",
        },
        {
            "instance": ApplyImpulseResponse(
                p=1.0, ir_path=os.path.join(DEMO_DIR, "ir"), leave_length_unchanged=True
            ),
            "num_runs": 1,
            "name": "ApplyImpulseResponseLeaveLengthUnchanged",
        },
        {
            "instance": AddShortNoises(
                sounds_path=os.path.join(DEMO_DIR, "short_noises"),
                min_absolute_noise_rms_db=-30,
                max_absolute_noise_rms_db=-10,
                noise_rms="absolute",
                min_time_between_sounds=2.0,
                max_time_between_sounds=4.0,
                burst_probability=0.4,
                min_pause_factor_during_burst=0.01,
                max_pause_factor_during_burst=0.95,
                min_fade_in_time=0.005,
                max_fade_in_time=0.08,
                min_fade_out_time=0.01,
                max_fade_out_time=0.1,
                p=1.0,
            ),
            "num_runs": 5,
            "name": "AddShortNoisesAbsolute",
        },
        {
            "instance": AddShortNoises(
                sounds_path=os.path.join(DEMO_DIR, "short_noises"),
                min_snr_in_db=0,
                max_snr_in_db=8,
                noise_rms="relative",
                min_time_between_sounds=2.0,
                max_time_between_sounds=4.0,
                burst_probability=0.4,
                min_pause_factor_during_burst=0.01,
                max_pause_factor_during_burst=0.95,
                min_fade_in_time=0.005,
                max_fade_in_time=0.08,
                min_fade_out_time=0.01,
                max_fade_out_time=0.1,
                p=1.0,
            ),
            "num_runs": 5,
            "name": "AddShortNoisesRelative",
        },
        {
            "instance": AddShortNoises(
                sounds_path=os.path.join(DEMO_DIR, "short_noises"),
                min_snr_in_db=0,
                max_snr_in_db=8,
                noise_rms="relative",
                min_time_between_sounds=2.0,
                max_time_between_sounds=4.0,
                burst_probability=0.4,
                min_pause_factor_during_burst=0.01,
                max_pause_factor_during_burst=0.95,
                min_fade_in_time=0.005,
                max_fade_in_time=0.08,
                min_fade_out_time=0.01,
                max_fade_out_time=0.1,
                signal_gain_in_db_during_noise=-100.0,
                p=1.0,
            ),
            "num_runs": 5,
            "name": "AddShortNoisesWithSignalGain",
        },
        {
            "instance": AddShortNoises(
                sounds_path=os.path.join(DEMO_DIR, "short_noises"),
                min_snr_in_db=0,
                max_snr_in_db=8,
                noise_rms="relative",
                min_time_between_sounds=1.0,
                max_time_between_sounds=2.0,
                burst_probability=0.4,
                min_pause_factor_during_burst=0.01,
                max_pause_factor_during_burst=0.95,
                min_fade_in_time=0.005,
                max_fade_in_time=0.08,
                min_fade_out_time=0.01,
                max_fade_out_time=0.1,
                noise_transform=OneOf(
                    [
                        TanhDistortion(min_distortion=0.8, max_distortion=0.99),
                        RoomSimulator(
                            calculation_mode="rt60",
                            min_target_rt60=2.2,
                            max_target_rt60=3.0,
                            leave_length_unchanged=False,
                        ),
                    ]
                ),
                p=1.0,
            ),
            "num_runs": 5,
            "name": "AddShortNoisesWithNoiseTransform",
        },
        {"instance": BandPassFilter(p=1.0), "num_runs": 5},
        {"instance": BandStopFilter(p=1.0), "num_runs": 5},
        {"instance": ClippingDistortion(p=1.0), "num_runs": 5},
        {"instance": Gain(min_gain_in_db=-6, max_gain_in_db=6, p=1.0), "num_runs": 5},
        {"instance": GainTransition(p=1.0), "num_runs": 5},
        {"instance": HighPassFilter(p=1.0), "num_runs": 5},
        {"instance": HighShelfFilter(p=1.0), "num_runs": 5},
        {"instance": LowPassFilter(p=1.0), "num_runs": 5},
        {"instance": LowShelfFilter(p=1.0), "num_runs": 5},
        {
            "instance": PitchShift(min_semitones=-4, max_semitones=4, p=1.0),
            "num_runs": 5,
        },
        {
            "instance": Lambda(
                transform=lambda samples, sample_rate: samples - 0.2,
                p=1.0,
            ),
            "num_runs": 1,
        },
        {"instance": Limiter(p=1.0), "num_runs": 5},
        {"instance": LoudnessNormalization(p=1.0), "num_runs": 5},
        {
            "instance": Mp3Compression(backend="lameenc", p=1.0),
            "num_runs": 5,
            "name": "Mp3CompressionLameenc",
        },
        {
            "instance": Mp3Compression(backend="pydub", p=1.0),
            "num_runs": 5,
            "name": "Mp3CompressionPydub",
        },
        {"instance": Normalize(p=1.0), "num_runs": 1},
        {
            "instance": Padding(mode="silence", pad_section="end", p=1.0),
            "num_runs": 5,
            "name": "PaddingSilenceEnd",
        },
        {
            "instance": Padding(mode="wrap", pad_section="end", p=1.0),
            "num_runs": 5,
            "name": "PaddingWrapEnd",
        },
        {
            "instance": Padding(mode="reflect", pad_section="end", p=1.0),
            "num_runs": 5,
            "name": "PaddingReflectEnd",
        },
        {
            "instance": Padding(mode="silence", pad_section="start", p=1.0),
            "num_runs": 5,
            "name": "PaddingSilenceStart",
        },
        {
            "instance": Padding(mode="wrap", pad_section="start", p=1.0),
            "num_runs": 5,
            "name": "PaddingWrapStart",
        },
        {"instance": PeakingFilter(p=1.0), "num_runs": 5},
        {"instance": PolarityInversion(p=1.0), "num_runs": 1},
        {"instance": Resample(p=1.0), "num_runs": 5},
        {"instance": Reverse(p=1.0), "num_runs": 1},
        {
            "instance": Compose(
                [RoomSimulator(p=1.0, leave_length_unchanged=True), Normalize(p=1.0)]
            ),
            "num_runs": 5,
            "name": "RoomSimulator",
        },
        {
            "instance": Compose([SevenBandParametricEQ(p=1.0), Normalize(p=1.0)]),
            "num_runs": 5,
            "name": "SevenBandParametricEQ",
        },
        {
            "instance": Shift(min_fraction=-0.5, max_fraction=0.5, fade=False, p=1.0),
            "num_runs": 5,
            "name": "ShiftWithoutFade",
        },
        {
            "instance": Shift(min_fraction=-0.5, max_fraction=0.5, fade=True, p=1.0),
            "num_runs": 5,
            "name": "ShiftWithShortFade",
        },
        {
            "instance": Shift(
                min_fraction=-0.5,
                max_fraction=0.5,
                rollover=False,
                fade=True,
                fade_duration=0.3,
                p=1.0,
            ),
            "num_runs": 5,
            "name": "ShiftWithoutRolloverWithLongFade",
        },
        {"instance": TanhDistortion(p=1.0), "num_runs": 5},
        {"instance": TimeMask(p=1.0), "num_runs": 5},
        {"instance": TimeStretch(min_rate=0.8, max_rate=1.25, p=1.0), "num_runs": 5},
        {"instance": Trim(p=1.0), "num_runs": 1},
        {
            "instance": Compose(
                [
                    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
                    SomeOf(
                        (0, 2),
                        [
                            TimeStretch(min_rate=0.8, max_rate=1.25, p=1.0),
                            PitchShift(min_semitones=-4, max_semitones=4, p=1.0),
                        ],
                    ),
                    Shift(min_fraction=-0.5, max_fraction=0.5, p=0.5),
                    OneOf([TanhDistortion(p=1.0), ClippingDistortion(p=1.0)], p=0.25),
                ]
            ),
            "num_runs": 10,
            "name": "BigCompose",
        },
        {"instance": AirAbsorption(p=1.0), "num_runs": 5},
    ]

    for sound_file_path in sound_file_paths:
        samples, sample_rate = load_sound_file(
            sound_file_path, sample_rate=None, mono=False
        )
        if len(samples.shape) == 2 and samples.shape[0] > samples.shape[1]:
            samples = samples.transpose()

        print(
            "Transforming {} with shape {}".format(
                sound_file_path.name, str(samples.shape)
            )
        )
        execution_times = {}

        for transform in tqdm(transforms):
            augmenter = transform["instance"]
            run_name = (
                transform.get("name")
                if transform.get("name")
                else transform["instance"].__class__.__name__
            )
            execution_times[run_name] = []
            for i in range(transform["num_runs"]):
                output_file_path = os.path.join(
                    output_dir,
                    "{}_{}_{:03d}.wav".format(sound_file_path.stem, run_name, i),
                )
                try:
                    with timer() as t:
                        augmented_samples = augmenter(
                            samples=samples, sample_rate=sample_rate
                        )
                    execution_times[run_name].append(t.execution_time)

                    if len(augmented_samples.shape) == 2:
                        augmented_samples = augmented_samples.transpose()

                    wavfile.write(
                        output_file_path, rate=sample_rate, data=augmented_samples
                    )
                except MultichannelAudioNotSupportedException as e:
                    print(e)

        for run_name in execution_times:
            if len(execution_times[run_name]) > 1:
                print(
                    "{:<32} {:.3f} s (std: {:.3f} s)".format(
                        run_name,
                        np.mean(execution_times[run_name]),
                        np.std(execution_times[run_name]),
                    )
                )
            else:
                print(
                    "{:<32} {:.3f} s".format(
                        run_name, np.mean(execution_times[run_name])
                    )
                )

NameError: name '__file__' is not defined